# Data Analysis

In [1]:
# Importing libraries
import yaml
import pymongo

In [2]:
# File function - Read dict from YAML file
def get_dict_from_yaml(yaml_path:str, encoding:str="utf-8") -> dict:
    result = dict()
    
    try:
        with open(yaml_path, mode="r", encoding=encoding) as file:
            yaml_file = file.read()
            result = yaml.load(yaml_file, Loader=yaml.FullLoader)
    except Exception as e:
        print(e)
        
    return result

In [3]:
# Read MongoDB login info
filepath = "config/mdb_setup.yaml"
mdb_login = get_dict_from_yaml(filepath)

In [4]:
# Create connetion to MongoDB
client  = pymongo.MongoClient(mdb_login['db_server'], mdb_login['db_port'])
db = client[mdb_login['db_name']]
coll = db[mdb_login['db_collection']]

In [5]:
# Query data
doc_list = []
for doc in coll.find({}):
    doc_list.append(doc)

print(len(doc_list))

0


<hr>
<a href="https://github.com/ansegura7/recsys-tweet-analysis">« Back to the repository.</a>